#Funcoes e Classes

In [ ]:
from itertools import chain, combinations
import os
################FUNCOES#######################

#Fazer as transicoes
def Ligacao_Vazio(Automato,estado):
  for e in Automato.estados:
    if e == estado:
      if len(estado.ligacoes['&']) != 0:
        return True
  return False

#Modelando o nome de cada estado
def Nome(nome):
  aux = '['
  if type(nome) == int:
    aux += str(nome)
  elif type(nome) == list:
    for l in nome:
      aux += str(l) + ', '
    remove = len(aux) - 2
    aux = aux[:remove]
  aux += ']'
  return aux

#Percorre os estados, para verificar ligações com o vazio 
def PegaDestinos(automato,estado,simbolo):
  aux = estado.ligacoes[simbolo]
  x = ''
  for i in aux:
    x += i.nome + ','

  for i in aux:
    if Ligacao_Vazio(automato,i):
      add = PegaDestinos(automato,i,'&')
      x += add
  
  return x  

def format_name(nome):
  return list(nome.replace(',',''))

#Transformando o nome lido em string para, para uma lista  
def string_para_lista(nome):
  nome = list(nome)
  lista = []
  for i in nome:
    if i !=  '[' and i != ']' and i != ',' and i != ' ':
      lista.append(int(i))
  return lista

#Função responsavel por procurar estados
def Procura(automato,nome):
  for i in automato.estados:
    if i.nome == nome:
      return i

#formatacao Uniao
def format_arco(conjunto):
  tam = len(conjunto)
  if tam > 1:
    uniao = "("
    i = 0
    for c in conjunto:
      uniao += str(c)
      i += 1
      if i < len(conjunto):
        uniao +=' U '
    uniao += ')'
  elif conjunto != []:
    uniao = conjunto[0]
  else:
    uniao = ''
  return uniao

#Criando Novo Automato
def Gerando_Automato(origem,qtd,Alfabeto):
    N = origem.qtd + qtd
    destino = Automato(estados = list(range(0,N)) ,alfabeto= Alfabeto)
    destino.qtd = N
    destino.inicial = destino.estados[0]
    destino.finais.add(destino.estados[N-1])
    destino.estados[0].inicial = True
    destino.estados[0].ligacoes['&'][0] = destino.estados[1]
    destino.estados[0].ligacoes_inv[destino.estados[1]][0] = '&'
    estados = list(origem.finais)
    for q in estados:
      if q.final :
        destino.estados[int(q.nome)].ligacoes['&'][0] = destino.estados[N-1]
        destino.estados[int(q.nome)].ligacoes_inv[destino.estados[N-1]][0] = '&'

    #Adicionando Transições
    for q in origem.estados:
      for l in q.ligacoes.keys():
        indice = int(q.nome)
        d = q.ligacoes[l]
        dstn = destino.estados[int(d[0].nome)]
        destino.estados[indice].ligacoes[l][0] = dstn

      for l in q.ligacoes_inv.keys():
        indice = int(q.nome)
        d = q.ligacoes_inv[l]
        if type(d) == list:
          d = format_arco(d)
        nome = int(l.nome)
        dstn = destino.estados[nome]  
        destino.estados[indice].ligacoes_inv[dstn][0] = d
        
    return destino

def pega_simbolo(item):
  if item[0] != '':
    return item[0]
  return ''

def pega_transicoes_destino(estado):
  destinos = []
  #print("Destinos de q",estado.nome,' : [',sep = '',end='')
  for q in estado.ligacoes_inv.keys():
    if estado.ligacoes_inv[q][0] != '':
      destinos.append(q)
  return destinos

def Remocao(Automato,qr):

  #print("Q = {",end='')
  #for i in Automato.estados:
    #print(i.nome,end = ', ')
  #print("}")
    
  #print("Ql = {",end='')
  destinos = pega_transicoes_destino(qr)
  Ql = []
  for ql in destinos:
    if ql != qr:
      Ql.append(ql)
      #print(ql.nome,end = ', ')
  #print("}")

  qs = Automato.estados[0]
  qa = Automato.estados[Automato.qtd - 1]

  #print("Qli = {",end='')
  Qli = []
  Qli.append(qs)
  #print(Qli[0].nome,end = ', ')
  for qli in Ql:
    if qli != qa:
      Qli.append(qli)
      #print(qli.nome,end = ', ')
  #print("}")
  
  #print("Qlj =  {",end='')
  Qlj = []
  for qlj in Ql:
    if qlj != qs:
      Qlj.append(qlj)
      #print(qlj.nome,end = ', ')
  #print("}")
  
  for qi in Qli:
    for qj in Qlj:
      #print("( q",qi.nome, "-> q", qj.nome + ")" , sep='', end = ' ') 
      qi.ligacoes_inv[qj] = ER_resultante(qi,qr,qj) 
      #print()
    
  for q in Automato.estados:
    if q != qr:
      q.ligacoes_inv.pop(qr)
      for i in q.ligacoes.keys():
        if qr in q.ligacoes[i]:
          q.ligacoes[i].remove(qr)
    
  Automato.estados.remove(qr)
  Automato.qtd -= 1
  
  return Automato

#PROTOTIPO DA FUNCAO PARA ECONTRAR AS EXPRESSOES REGULARES
def ER_resultante(qi,qr,qj):
    
    R1 = qi.ligacoes_inv[qr] # qi,qr
    R2 = qr.ligacoes_inv[qr] # qr,qr
    R3 = qr.ligacoes_inv[qj] # qr,qj
    R4 = qi.ligacoes_inv[qj] # qi,aj
    Ex = ''

    if type(R1) == list and R1 != []:
      R1 = R1[0]
    if type(R2) == list and R2 != []:
      R2 = R2[0]
    if type(R3) == list and R3 != []:
      R3 = R3[0]
    if type(R4) == list and R4 != []:
      R4 = R4[0]

    if R1 != '&' and R1 != '':
      #print("R1 = ", R1, end = ' | ')
      Ex +=  str(R1) 
    if R2 != '&' and R2 != '':
      #print("R2 = ", R2, end = ' | ')
      Ex +=  str(R2) + "*" 
    if R3 != '&' and R3 != '':
      #print("R3 = ", R3, end = ' | ')
      Ex +=  str(R3) 
    if R4 != '' and  Ex != "&":
      #print("R4 = ", R4, end = ' | ')
      Ex += ' U ' + str(R4)
    
    #print(" Expressao = ", Ex, end = '  ')
    if Ex == '':
      return ['']
    else:
      return "(" + Ex + ")"

In [ ]:
class Automato:
  '''
  Classe Para Reprezentar Autômatos
  Inicialização de um objeto da classe Automato
        Parametros:
            * estados ---> Ou Um vetor contendo os nomes dos estados. Ex: [1,2,3];
                |--------> Ou Um inteiro indicando a quantidade de estados;
            * inicial -----> Indica o Estado Inicial do Automato (objeto da classe Estado);
            * finais ------> Conjunto de Estados finais (objetos da classe Estado);
                |--------> Caso None, é criado um conjunto Vazio;
            * alfabeto ---> Conjunto representante do alfabeto do Automato;
  
  '''
  def __init__(self, estados = 0, inicial=None, finais=None, alfabeto=None):
    
    ######### Verifica o que foi passado para estados #########
    self.estados = []
    if type(estados) == list:
      self.qtd = len(estados)
      for n in estados:
        if type(n) == int or type(n) == str:
          self.estados.append(Estado(str(n)))
        else:
          self.estados.append(n)
    elif type(estados) == int:
      self.qtd = estados
      for n in range(1,estados+1):
        self.estados.append(Estado(str(n)))

    ######### Seta o estado Inicial ############
    self.inicial = inicial

    ######### Seta os estados Finais ###########
    if finais == None:
      self.finais = set()
    else:
      self.finais = finais
    self.alfabeto = alfabeto
    self.Criar_Transicoes()

  def Criar_Transicoes(self):
    '''
    Função para inicializar as transições de cada estado com cada simbolo do alfabeto
    '''
    for i in self.estados:
      for a in self.alfabeto:
        i.ligacoes[a] = ['']
    
    for i in self.estados:
      for e in self.estados:
        i.ligacoes_inv[e] = ['']

  def Transicao(self, e1,e2):
    '''
    Função que retorna a lista de simbolos da transição entre os dois estados mencionados
    '''    
    return e1.ligacoes_inv[e2]

  def Mostra_Estados(self):
    '''
    Função de Visualização dos Estados presentes no Autômato
    '''

    print("- - - - - - - - - - - - - - - -")
    print("ESTADOS: ",end='')
    i = 0
    for e in self.estados:
      print('q',e.nome,sep='', end = ' ')
      i += 1
      if i < len(self.estados):
        print("|",end=' ')
      else:
        print()
    print("- - - - - - - - - - - - - - - -")

  def Mostra_ligacoes(self):
    print("\n- - - - - - - - - - - - - - - -\n")
    for q in self.estados:
      print("Estado: ",q.nome)
      for s in q.ligacoes_inv.keys():
        if q.ligacoes_inv[s] != '':
          print("----> Vai para ", s.nome, " com o simbolo : ", q.ligacoes_inv[s])
      print("\n- - - - - - - - - - - - - - - -\n")

  def Mostra_Automato(self,tipo='afnd'):
    '''
    Função de Visualização do Autômato por completo

    Parâmetros:
        * tipo ---> String Indicando se é AFND ou AFD (opcional);
    '''

    print("- - - - - - - - - - - AUTOMATO - - - - - - - - - - - -")
    print("ALFABETO: ", self.alfabeto)
    self.Mostra_Estados()
    print("ESTADO INICIAL:",end='') 
    if self.inicial:
      print(" q",self.inicial.nome,sep='')
    else:
      print(" NAO DEFINIDO")

    print("- - - - - - - - - - - - - - - -")

    if type(self.finais) == set:
      if len(self.finais) != 0:
        if len(self.finais) > 1:
          print("ESTADOS FINAIS: ",end='')
        elif len(self.finais) == 1:
          print("ESTADO FINAL: ",end='')
        i = 0
        for f in self.finais:
          print("q",f.nome, sep='',end='')
          i+= 1
          if i < len(self.finais):
            print(" | ",end='')
          else:
            print()
      else:
        print("ESTADOS FINAIS: NAO DEFINIDOS")
    else:
      print("ESTADO FINAL: q",self.finais.nome, sep='')

    print("\n- - - - - - - - - - - - - - - -\n")
    
    self.Mostra_ligacoes()

#Class Estado:
class Estado:
  '''
  Classe para representar um Estado
  
  Parametros:
          * nome ---> String indicando o nome do Estado. Ex: '1';
              |------> Caso nada seja passado, é criado um estado com nome '';
          * inicial -----> Indica se o Estado é inicial ou não. Admite True ou False como entrada;
              |------> Caso nada seja passado, o estado é colocado como não inicial;
          * final ------> Indica se o Estado é final ou não. Admite True ou False como entrada;
              |------> Caso nada seja passado, o estado é colocado como não final;
          * ligacoes --> Um dicionario(dict) que representa todas as ligações que o estado realiza;
              |------> Caso nada seja passado, é criado um dicionário vazio;
    '''
  def __init__(self,nome="",inicial=False,final=False,ligacoes={}):
    
    self.nome = nome
    self.inicial = inicial
    self.final = final
    self.ligacoes = dict()
    self.ligacoes_inv = dict()
  def Mostra_Estado(self):
    ''' 
    Função para visualização do Estado e suas ligacoes
    '''
    print("ESTADO ATUAL: q",self.nome,sep='')
    for a in self.ligacoes.keys():
      print("+ ",a," ---> ",end='')
      for i in self.ligacoes[a]:
       print("q",i.nome, sep='',end = ' | ')
      print()

  def Criar_Ligacoes(self,proximo_estado="",simbolo=""): 
    '''
    Função para criar as ligações que o estado realiza
    Parâmetros:
      * proximo_estado ---> Objeto da classe estado ao qual o atual estado é ligado dado determinado simbolo
      * simbolo ----------> Simbolo do alfabeto que com o qual será feita a ligação
    '''
    if self.ligacoes[simbolo][0] == '':
      self.ligacoes[simbolo][0] = proximo_estado
    else:
      self.ligacoes[simbolo].append(proximo_estado)

    if self.ligacoes_inv[proximo_estado][0] == '':
      self.ligacoes_inv[proximo_estado][0] = simbolo
    else:
      self.ligacoes_inv[proximo_estado].append(simbolo)

#Criacao Do AFD

In [ ]:
#CRIACAO DO ALFABETO:
print("Digite o alfabeto do autômato AFND de forma continua")
print("Exemplo: Entrada '01'") 
a = input()
Alfabeto = set()
for i in a:
  Alfabeto.add(i)
Alfabeto = sorted(Alfabeto)
print("Alfabeto: ",Alfabeto)

#Lendo e definindo a quantidade de estados do AFD
print("Digite a quantidade de estados terá seu automato: ")
tam = int(input())
AFD = Automato(estados=tam,alfabeto=Alfabeto)

#Lendo e definindo os estados iniciais do AFD
AFD.Mostra_Estados()
print("Informe qual dos estados listados acima será o estado inicial:")
confirma = 0
while confirma == 0:  
  valor = input()
  for i in AFD.estados:
    if i.nome == valor[1:]:
      i.inicial = True
      AFD.inicial = i
      confirma = 1
      break

#Lendo e definindo os estados finais do AFD
AFD.Mostra_Estados()
print("Informe qual dos estados listados acima sera o estado final:")
valores = input()
valores = valores.split('q')
if '' in valores:
  indice = valores.index('')
  valores.pop(indice)
for i in valores:
  print("Estado final: q",i,sep='',end='')
  if int(i) <= tam:
    for e in AFD.estados:
      if e.nome == i:
        e.final = True
        AFD.finais.add(e)
    print(" - - - Adicionado !")
  else:
    print(" - - - Invalido! !")
  
print("\n- - - - - - - - TRANSIÇÕES - - - - - - -\n")
AFD.Mostra_Estados()

#PARA CADA ESTADO
for e in AFD.estados:
  #PARA CADA SIMBOLO DO ALFABETO
  for a in Alfabeto:
    confirma = 0
    print(" -> Estado q",e.nome," com o simbolo \'",a,"\' vai para qual estado?",sep='')
    while confirma == 0:
      quant = input()
      if len(quant) < 3:
        confirma = 1  

    if quant != '':
      quant = quant.split('q')
      if '' in quant:
        indice = quant.index('')
        quant.pop(indice)
      for q in quant:
        index = AFD.estados.index(e)
        for eadd in AFD.estados:
          if eadd.nome == q:
            AFD.estados[index].Criar_Ligacoes(eadd,a)
    #e.Mostra_Estado()
    print()
  print("- - - - - - - - - - - - - - - - - - - - - - ")
  
os.system('cls') or None
print("\n--------- MOSTRANDO O AUTÔMATO AFND ---------\n")
AFD.Mostra_Automato()

Digite o alfabeto do autômato AFND de forma continua
Exemplo: Entrada '01'
ab
Alfabeto:  ['a', 'b']
Digite a quantidade de estados terá seu automato: 
2
- - - - - - - - - - - - - - - -
ESTADOS: q1 | q2 
- - - - - - - - - - - - - - - -
Informe qual dos estados listados acima será o estado inicial:
q1
- - - - - - - - - - - - - - - -
ESTADOS: q1 | q2 
- - - - - - - - - - - - - - - -
Informe qual dos estados listados acima sera o estado final:
q2
Estado final: q2 - - - Adicionado !

- - - - - - - - TRANSIÇÕES - - - - - - -

- - - - - - - - - - - - - - - -
ESTADOS: q1 | q2 
- - - - - - - - - - - - - - - -
 -> Estado q1 com o simbolo 'a' vai para qual estado?
q1

 -> Estado q1 com o simbolo 'b' vai para qual estado?
q2

- - - - - - - - - - - - - - - - - - - - - - 
 -> Estado q2 com o simbolo 'a' vai para qual estado?
q2

 -> Estado q2 com o simbolo 'b' vai para qual estado?
q2

- - - - - - - - - - - - - - - - - - - - - - 

--------- MOSTRANDO O AUTÔMATO AFND ---------

- - - - - - - - - - - 

# Criando AFNG




In [ ]:
#Atualizando Alfabeto
Alfabeto = set(Alfabeto)
Alfabeto.add('&')
Alfabeto = list(sorted(Alfabeto))

AFNG = Gerando_Automato(AFD,2,Alfabeto)
AFNG.Mostra_Automato()

- - - - - - - - - - - AUTOMATO - - - - - - - - - - - -
ALFABETO:  ['&', 'a', 'b']
- - - - - - - - - - - - - - - -
ESTADOS: q0 | q1 | q2 | q3 
- - - - - - - - - - - - - - - -
ESTADO INICIAL: q0
- - - - - - - - - - - - - - - -
ESTADO FINAL: q3

- - - - - - - - - - - - - - - -


- - - - - - - - - - - - - - - -

Estado:  0
----> Vai para  0  com o simbolo :  ['']
----> Vai para  1  com o simbolo :  ['&']
----> Vai para  2  com o simbolo :  ['']
----> Vai para  3  com o simbolo :  ['']

- - - - - - - - - - - - - - - -

Estado:  1
----> Vai para  0  com o simbolo :  ['']
----> Vai para  1  com o simbolo :  ['a']
----> Vai para  2  com o simbolo :  ['b']
----> Vai para  3  com o simbolo :  ['']

- - - - - - - - - - - - - - - -

Estado:  2
----> Vai para  0  com o simbolo :  ['']
----> Vai para  1  com o simbolo :  ['']
----> Vai para  2  com o simbolo :  ['(a U b)']
----> Vai para  3  com o simbolo :  ['&']

- - - - - - - - - - - - - - - -

Estado:  3
----> Vai para  0  com o simbolo :  ['']


#Removendo Estados e Obtendo ER

In [ ]:
for i in range(AFD.qtd):
  AFNG = Remocao(AFNG,AFNG.estados[1])

ER = AFNG.Transicao(AFNG.estados[0],AFNG.estados[1])

print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
print("\n\nEXPRESSAO REGULAR: ", ER)
print("\n\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - \n\n")
print("AUTOMATO RESULTANTE: ")
AFNG.Mostra_Automato()

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


EXPRESSAO REGULAR:  ((a*b)(a*b U (a U b))*)


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


AUTOMATO RESULTANTE: 
- - - - - - - - - - - AUTOMATO - - - - - - - - - - - -
ALFABETO:  ['&', 'a', 'b']
- - - - - - - - - - - - - - - -
ESTADOS: q0 | q3 
- - - - - - - - - - - - - - - -
ESTADO INICIAL: q0
- - - - - - - - - - - - - - - -
ESTADO FINAL: q3

- - - - - - - - - - - - - - - -


- - - - - - - - - - - - - - - -

Estado:  0
----> Vai para  0  com o simbolo :  ['']
----> Vai para  3  com o simbolo :  ((a*b)(a*b U (a U b))*)

- - - - - - - - - - - - - - - -

Estado:  3
----> Vai para  0  com o simbolo :  ['']
----> Vai para  3  com o simbolo :  ['']

- - - - - - - - - - - - - - - -

